In [26]:
import os
import numpy as np
import pandas as pd
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler  

import matplotlib.pyplot as plt
%matplotlib inline

print(f"Your version of Pytorch is {torch.__version__}.")

from dotenv import load_dotenv
load_dotenv(r'.env')
data_dir = os.environ.get('DATA_DIR')

Your version of Pytorch is 2.1.1+cpu.


In [27]:
# If a GPU is available, use it
# Pytorch uses an elegant way to keep the code device agnostic
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False

print(device)

cpu


In [28]:
data = pd.read_csv(data_dir + "/sign_mnist_train.csv")

In [29]:
np.unique(data['label'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)

In [30]:
def adjust_class_labels(label):
   
    if label >= 10:
        label -= 1
    
    return label

# Adjusting the class labels (training data)
data["label"] = data["label"].apply(adjust_class_labels)

In [31]:
np.unique(data['label'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int64)

In [32]:
X_train, X_val, y_train, y_val = train_test_split(data.iloc[:,1:], data['label'], test_size=0.1, stratify=data['label'], random_state=21)

In [33]:
X_train = X_train.values.reshape(-1,28,28,1)
X_val = X_val.values.reshape(-1,28,28,1)

#scaling 
X_train=X_train/255.0
X_val=X_val/255.0

In [34]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from torch import nn
from torch.nn import functional as f
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

In [35]:
#convert the lables to categorical data using one-hot encoding
y_train_one_hot=to_categorical(y_train)  

y_val_one_hot=to_categorical(y_val)  

In [36]:
class StratifiedSampler:

    def __init__(self, class_vector: torch.Tensor, batch_size: int, seed: int):
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector
        self.seed = seed

    def gen_sample_array(self):
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5, random_state=self.seed)
        X = torch.randn(self.class_vector.size(0),2).numpy()
        y = self.class_vector.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [37]:
from keras.preprocessing.image import ImageDataGenerator

In [38]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.2,
        width_shift_range=0.1,
        height_shift_range=0.1)

datagen.fit(X_train.reshape(-1,28,28,1))
sampler = StratifiedSampler(class_vector=torch.tensor(y_train.values), batch_size=32, seed=1)
x_train, y_train = next(datagen.flow(X_train.reshape(-1,28,28,1), y_train, batch_size=y_train.size))
train_loader = DataLoader(list(zip(x_train, y_train)), batch_size=32, sampler=sampler)
valid_loader = DataLoader(list(zip(X_val, y_val)), batch_size=y_val.size)
#test_loader = DataLoader(list(zip(X_test, y_test)), batch_size=y_test.size)

In [39]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.c1 = nn.Conv2d(in_channels=1, out_channels=45, kernel_size=3, stride=1, padding=1)
        self.batch_norm1= nn.BatchNorm2d(45)
        self.c2 = nn.Conv2d(in_channels=45, out_channels=55, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(p=0.2)
        self.batch_norm2 = nn.BatchNorm2d(55)
        self.flattent = nn.Flatten(start_dim=1)
        self.fc = nn.Linear(in_features=2695, out_features=24)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = f.relu(self.c1(x))
        x = self.batch_norm1(x)
        x = self.max_pool(x)
        x = f.relu(self.c2(x))
        x = self.dropout(x)
        x = self.batch_norm2(x)
        x = self.max_pool(x)
        x = self.flattent(x)
        x = self.softmax(self.fc(x))
        return x

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [42]:
EPOCHS = 5 
BATCH_SIZE = 16
LEARNING_RATE = 0.0007

In [43]:
NUM_FEATURES = 784
NUM_CLASSES = 24

model = CNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

CNN(
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c1): Conv2d(1, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm1): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c2): Conv2d(45, 55, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (batch_norm2): BatchNorm2d(55, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (flattent): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=2695, out_features=24, bias=True)
  (softmax): Softmax(dim=1)
)


In [45]:
for epoch in range(5):
    model.train()
    for X, y in tqdm(train_loader):
        optimizer.zero_grad()
        X = X.reshape(-1,1, 28,28).to(device)
        y = y.to(device)
        out = model(X)
        loss = criterion(out, f.one_hot(y, 24).float())
        loss.backward()
        optimizer.step()
    model.eval()
    val_X, val_y = next(iter(valid_loader))
    val_X = val_X.to(device)
    val_y = val_y.to(device)
    print(f'Validation accuracy after epoch {epoch + 1}: '
          f'{accuracy_score(val_y.cpu(), torch.argmax(model(val_X), dim=1).cpu())}')

100%|██████████| 773/773 [00:18<00:00, 40.92it/s]


RuntimeError: Input type (double) and bias type (float) should be the same

In [18]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [24]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [25]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()

    # VALIDATION    
    with torch.no_grad():
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in valid_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()

        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        loss_stats['val'].append(val_epoch_loss/len(valid_loader))
        accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
        accuracy_stats['val'].append(val_epoch_acc/len(valid_loader))
        
        print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')


Begin training.


  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [45, 1, 3, 3], expected input[32, 28, 28, 1] to have 1 channels, but got 28 channels instead

In [8]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
#X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
#X_test, y_test = np.array(X_test), np.array(y_test)

In [9]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index] # changement de type ici
        
    def __len__ (self):
        return len(self.X_data)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
#test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

In [10]:
EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 0.0007

In [15]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE)
valid_loader = DataLoader(dataset=val_dataset, batch_size=1)
#test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [12]:
class BasicNet(nn.Module):
    """Affordable convolutions for the people."""

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 16, (3, 3), padding=1, stride=1)
        self.conv2 = nn.Conv2d(16, 64, (5, 5), padding=2)

        self.reshape_size = 64*7*7
        self.fc = nn.Linear(64*7*7, 10)

    def forward(self, xin):
        # xin is [batch_size, channels, heigth, width] = [bs, 1, 28, 28]

        x = F.relu(self.conv1(xin))
        # [bs, 16, 28, 28]
        x = F.max_pool2d(x, 2)
        # x is [bs, 16, 14, 14]

        x = F.relu(self.conv2(x))
        # x [bs, 64, 14, 14]
        x = F.max_pool2d(x, 2)
        # x is [bs, 64, 7, 7]

        #TODO : We now flatten x. What shape should we give it ?
        x = x.view(-1, self.reshape_size)
        x = self.fc(x)
        return x


In [13]:
# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='mean')

# spot to save your learning curves, and potentially checkpoint your models
savedir = 'results'
if not os.path.exists(savedir):
    os.makedirs(savedir)

In [14]:
def train(model, train_loader, optimizer, epoch):
    """Perform one epoch of training."""
    model.train()

    # we initialize our kpis to keep track of across batches
    avg_loss = 0.
    avg_accuracy = 0.

    for batch_idx, (inputs, target) in enumerate(train_loader):
        inputs, target = inputs.to(device), target.to(device)

        # TODO: code this training loop. Remember lab 8 (or take a quick look)
        # A FAIRE: Implementer cette boucle d'entrainement. Rappelez vous du lab 8 (ou jettez y un coup d'oeil)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        avg_accuracy += (output.argmax(-1) == target).to(torch.float).mean().item()
        avg_loss += loss.item()

    avg_loss /= len(train_loader)
    avg_accuracy /= len(train_loader)

    print(f'Train - Loss: {avg_loss:.5f}, Accuracy: {avg_accuracy:.5f}')

    return avg_loss, avg_accuracy

In [16]:
def evaluate(model, test_loader, evaluation_type='Validation'):
    """Evaluate the model by doing one pass over a dataset"""
    model.eval()

    # we initialize our kpis to keep track of across batches
    avg_loss = 0.
    avg_accuracy = 0.

    with torch.no_grad(): # save some computations

        for inputs, target in test_loader:
            inputs, target = inputs.to(device), target.to(device)
            output = model(inputs)
            loss = loss_fn(output, target)

            # TODO: code the evaluation loop
            avg_accuracy += (output.argmax(-1) == target).to(torch.float).mean().item()
            avg_loss += loss.item()

    avg_loss /= len(test_loader)
    avg_accuracy /= len(test_loader)

    print(f'{evaluation_type} - Loss: {avg_loss:.5f}, Accuracy: {avg_accuracy:.5f}')

    return avg_loss, avg_accuracy

In [17]:
model = BasicNet().to(device)

lr = 0.0005
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

results_basic = {'name':'basic',
                 'lr': lr,
                 'train_loss': [],
                 'train_accuracy':[],
                 'valid_loss': [],
                 'valid_accuracy':[]}
savefile = os.path.join(savedir, results_basic['name']+str(results_basic['lr'])+'.pkl' )

max_patience = 15
patience = 0
best_acc = None
best_epoch = None
best_model_state = None
for epoch in range(1, 120):

    print(f"-- Epoch {epoch} -- (patience: {patience}/{max_patience})")
    # train
    train_loss, train_acc = train(model, train_loader, optimizer, epoch)

    # validate
    valid_loss, valid_acc = evaluate(model, valid_loader, evaluation_type='Validation')

    print("")  # new line

    # save results every epoch
    results_basic['train_loss'].append(train_loss)
    results_basic['train_accuracy'].append(train_acc)
    results_basic['valid_loss'].append(valid_loss)
    results_basic['valid_accuracy'].append(valid_acc)

    # TODO early stopping
    if best_acc is None or valid_acc >= best_acc:
        patience = 0
        best_acc = valid_acc
        best_epoch = epoch
        best_model_state = model.state_dict()
    else:
        patience += 1

    with open(savefile, 'wb') as fout:
        pickle.dump(results_basic, fout)

    if patience > max_patience:
        break

print(f"Exited at epoch: {epoch}, best_accuracy: {best_acc}")

-- Epoch 1 -- (patience: 0/15)


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [16, 784]